In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import sqlite3
import os
import re

print(os.getcwd())

/home/ssg145/final


# Download & Clean Each Data Set

We have 3 different data sets that we are using for the project.
1. Coffee Quality Data
- From the Coffee Quality Data we extracted the Country Name, No. of Bags, Harvest Year and Bag Weight.
2. Meat Production Data
- From the Meat Dataset we focused on analyzing Country, Year, and Production in tonns
3. CO2 Emmisions
- From the CO2 Emmisions dataset we focused on the Country, Year, and the CO2 Emissions in kiloton (kt)
4. Milk Production Dataset
- From the Milk Production Dataset 

## Coffee Quality Dataset

In [3]:
project_dir = os.getcwd()

# Construct the full path to the CSV file
file_path = os.path.join(project_dir, 'coffee.csv')

# Read the CSV file
coffee_df = pd.read_csv(file_path)

#Rename the country column
coffee_df = coffee_df.rename(columns={'Country of Origin': 'Country'})

#coffee_df['Bag Weight'] = coffee_df['Bag Weight'].str.replace('kg', '').astype(float)
coffee_df['Bag Weight'] = coffee_df['Bag Weight'].str.replace(r'\s*[Kk][Gg]', '', regex=True).astype(float)


#CLEANING UP THE DATA (getting rid of unneccessary info ie. columns)
dimensions = coffee_df.shape
print("Dimensions before Cleaning:", dimensions)
print(coffee_df.head(6))

coffee_df = coffee_df[["Country", "Number of Bags", "Bag Weight", "Harvest Year"]]#Taking only the columns we need

dimensions = coffee_df.shape
print("\nDimensions after Cleaning:",dimensions)
print(coffee_df.head(6))
coffee_df.to_csv('output.csv', index=False)

#May want to drop all the columns except the country name, no. of bags, harvest year and bag weight.

Dimensions before Cleaning: (207, 41)
   Unnamed: 0  ID     Country                 Farm Name  \
0           0   0    Colombia          Finca El Paraiso   
1           1   1      Taiwan  Royal Bean Geisha Estate   
2           2   2        Laos        OKLAO coffee farms   
3           3   3  Costa Rica                 La Cumbre   
4           4   4    Colombia           Finca Santuario   
5           5   5   Guatemala                 La Colina   

                                Lot Number                           Mill  \
0                               CQU2022015               Finca El Paraiso   
1  The 2022 Pacific Rim Coffee Summit,T037       Royal Bean Geisha Estate   
2  The 2022 Pacific Rim Coffee Summit,LA01  oklao coffee processing plant   
3                               CQU2022017        La Montana Tarrazu MIll   
4                               CQU2023002                Finca Santuario   
5  The 2022 Pacific Rim Coffee Summit,GT02                  Dinámica Café   

  ICO Nu

### Null/Missing Values

In [4]:
#Finding Missing Values
#Column 1
missing_countries = coffee_df["Country"].isnull().sum()
print("Amount of Missing Values for Column 1:", missing_countries)

#Column 2
missing_bags = coffee_df["Number of Bags"].isnull().sum()
print("Amount of Missing Values for Column 2:", missing_bags)

#Column 3
missing_weight = coffee_df["Bag Weight"].isnull().sum()
print("Amount of Missing Values for Column 3:", missing_weight)

#Column 4
missing_year = coffee_df["Harvest Year"].isnull().sum()
print("Amount of Missing Values for Column 4:", missing_year)

#Finding Null Values

Amount of Missing Values for Column 1: 0
Amount of Missing Values for Column 2: 0
Amount of Missing Values for Column 3: 0
Amount of Missing Values for Column 4: 0


## Meat Dataset

In [5]:
project_dir = os.getcwd()

# Construct the full path to the CSV file
file_path = os.path.join(project_dir, 'meat.csv')
meat_df = pd.read_csv(file_path)
#df = pd.read_csv('/home/ssg145/final/meat.csv')
initial_size = meat_df.shape[0]

dimensions = meat_df.shape
print("Dimensions before Cleaning:", dimensions)
print(meat_df.head(6))

#CLEANING UP DATA
meat_df = meat_df.drop(columns=['Code'])
#print(df.head(6))

meat_df = meat_df.rename(columns={'Meat, total | 00001765 || Production | 005510 || tonnes': 'Production amt'})

dimensions = meat_df.shape
print("\nDimensions after Cleaning:", dimensions)
print(meat_df.head(6))


Dimensions before Cleaning: (14382, 4)
               Country Code  Year  \
0              Bahamas  BHS  1961   
1               Brunei  BRN  1961   
2                Qatar  QAT  1961   
3        Faroe Islands  FRO  1961   
4               Tuvalu  TUV  1961   
5  Antigua and Barbuda  ATG  1961   

   Meat, total | 00001765 || Production | 005510 || tonnes  
0                                             1260.7        
1                                             1289.8        
2                                             1769.2        
3                                                0.0        
4                                               30.0        
5                                              395.0        

Dimensions after Cleaning: (14382, 3)
               Country  Year  Production amt
0              Bahamas  1961          1260.7
1               Brunei  1961          1289.8
2                Qatar  1961          1769.2
3        Faroe Islands  1961             0.0
4         

### Null/Missing Values

In [6]:
#Finding Missing Values
#Column 1
missing_countries = meat_df["Country"].isnull().sum()
print("Amount of Missing Values for Column 1:", missing_countries)

#Column 2
missing_year = meat_df["Year"].isnull().sum()
print("Amount of Missing Values for Column 2:", missing_year)

#Column 3
missing_production = meat_df["Production amt"].isnull().sum()
print("Amount of Missing Values for Column 3:", missing_production)

#Finding Null Values: there are country values that are continents or low income/high income countries
print(meat_df.shape)
bad_values = ["Asia", "North America", "Africa", "South America", "Europe", "Antartica", "World", "Lower-middle-income countries", "Upper-middle-income countries", "High-income countries", "Low-income countries"]
meat_df = meat_df[~meat_df["Country"].str.contains("|".join(bad_values), case = False, na = False)]
print(meat_df.shape)

Amount of Missing Values for Column 1: 0
Amount of Missing Values for Column 2: 0
Amount of Missing Values for Column 3: 0
(14382, 3)
(12429, 3)


## CO2 Emmissions Dataset

In [7]:
#df = pd.read_csv('./final/co2_country.csv')
project_dir = os.getcwd()

# Construct the full path to the CSV file
file_path = os.path.join(project_dir, 'co2_country.csv')
df_co2 = pd.read_csv(file_path)

dimensions = df_co2.shape
print("Dimensions before Cleaning:", dimensions)
print(df_co2.head(6))

#Rename the country column 
df_co2 = df_co2.rename(columns={'country_name': 'Country'})
df_co2 = df_co2.rename(columns={'value': 'co2_value'})

#Clean the data (get rid of the country codes)
df_co2 = df_co2.iloc[:, 1:]

dimensions = df_co2.shape
print("\nDimensions before Cleaning:", dimensions)
print(df_co2.head(6))

Dimensions before Cleaning: (13953, 4)
  country_code country_name  year      value
0          ABW        Aruba  1960  11092.675
1          ABW        Aruba  1961  11576.719
2          ABW        Aruba  1962  12713.489
3          ABW        Aruba  1963  12178.107
4          ABW        Aruba  1964  11840.743
5          ABW        Aruba  1965  10623.299

Dimensions before Cleaning: (13953, 3)
  Country  year  co2_value
0   Aruba  1960  11092.675
1   Aruba  1961  11576.719
2   Aruba  1962  12713.489
3   Aruba  1963  12178.107
4   Aruba  1964  11840.743
5   Aruba  1965  10623.299


### Null/Missing Values

In [8]:
#Finding Missing Values
#Column 1
missing_countries = df_co2["Country"].isnull().sum()
print("Amount of Missing Values for Column 1:", missing_countries)

#Column 2
missing_year = df_co2["year"].isnull().sum()
print("Amount of Missing Values for Column 2:", missing_year)

#Column 3
missing_value = df_co2["co2_value"].isnull().sum()
print("Amount of Missing Values for Column 3:", missing_value)


Amount of Missing Values for Column 1: 0
Amount of Missing Values for Column 2: 0
Amount of Missing Values for Column 3: 0


## Milk Production Dataset

In [9]:
project_dir = os.getcwd()

# Construct the full path to the CSV file
file_path = os.path.join(project_dir, 'milk.csv')
milk_df = pd.read_csv(file_path)
#df = pd.read_csv('/home/ssg145/final/meat.csv')
initial_size = milk_df.shape[0]

#print(df.head(6))

#Rename the country column
milk_df = milk_df.rename(columns={'Milk | 00001780 || Production | 005510 || tonnes': 'Milk amt'})
milk_df = milk_df.rename(columns={'Entity': 'Country'})

#CLEANING UP DATA
dimensions = milk_df.shape
print("Dimensions before Cleaning:", dimensions)
print(milk_df.head(6))
milk_df = milk_df.drop(columns=['Code'])



dimensions = milk_df.shape
print("\nDimensions after Cleaning:", dimensions)
print(milk_df.head(6))


Dimensions before Cleaning: (13615, 4)
       Country Code  Year  Milk amt
0  Afghanistan  AFG  1961  574900.0
1  Afghanistan  AFG  1962  576700.0
2  Afghanistan  AFG  1963  646700.0
3  Afghanistan  AFG  1964  659900.0
4  Afghanistan  AFG  1965  720900.0
5  Afghanistan  AFG  1966  750500.0

Dimensions after Cleaning: (13615, 3)
       Country  Year  Milk amt
0  Afghanistan  1961  574900.0
1  Afghanistan  1962  576700.0
2  Afghanistan  1963  646700.0
3  Afghanistan  1964  659900.0
4  Afghanistan  1965  720900.0
5  Afghanistan  1966  750500.0


### Null/Missing Values

In [10]:
#Finding Missing Values
#Column 1
missing_countries = milk_df["Country"].isnull().sum()
print("Amount of Missing Values for Column 1:", missing_countries)

#Column 2
missing_year = milk_df["Year"].isnull().sum()
print("Amount of Missing Values for Column 2:", missing_year)

#Column 3
missing_value = milk_df["Milk amt"].isnull().sum()
print("Amount of Missing Values for Column 3:", missing_value)


Amount of Missing Values for Column 1: 0
Amount of Missing Values for Column 2: 0
Amount of Missing Values for Column 3: 0


In [11]:
#Steps 
#Check for null/missing values 
#average the country values 
#join the datasets 
#Perform analysis (linear regression) 
#Create a prediction model 


### Join Datasets Using SQL

### Perform Analysis

### Prediction Model

In [12]:
#df_merged = pd.merge(df1, df2, on='ID')

# Example: Merging four DataFrames on 'Country'
df_merged = pd.merge(coffee_df, meat_df, on='Country', how='inner')
df_merged = pd.merge(df_merged, df_co2, on='Country', how='inner')
df_merged = pd.merge(df_merged, milk_df, on='Country', how='inner')

print(df_merged.head(6))

dimensions = df_merged.shape
print(dimensions)

    Country  Number of Bags  Bag Weight Harvest Year  Year_x  Production amt  \
0  Colombia               1        35.0  2021 / 2022    1961        442045.0   
1  Colombia               1        35.0  2021 / 2022    1961        442045.0   
2  Colombia               1        35.0  2021 / 2022    1961        442045.0   
3  Colombia               1        35.0  2021 / 2022    1961        442045.0   
4  Colombia               1        35.0  2021 / 2022    1961        442045.0   
5  Colombia               1        35.0  2021 / 2022    1961        442045.0   

   year  co2_value  Year_y   Milk amt  
0  1960  16409.825    1961  1762000.0  
1  1960  16409.825    1962  1784885.0  
2  1960  16409.825    1963  1833000.0  
3  1960  16409.825    1964  1860000.0  
4  1960  16409.825    1965  1973000.0  
5  1960  16409.825    1966  2020000.0  
(28501440, 10)


In [13]:
#Perform Analysis
x= df_merged[['Bag Weight', 'Production amt', 'Milk amt']]
y= df_merged['co2_value']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(x_train, y_train)
print("Coefficient (Slope):", model.coef_)
print("Intercept:", model.intercept_)



Coefficient (Slope): [-2.51160916  0.00669307  0.00578822]
Intercept: 30003.445504692023


In [22]:


#df1 = pd.DataFrame(data1)
#df2 = pd.DataFrame(data2)

# Create SQLite database in memory
conn = sqlite3.connect(':memory:')

# Write DataFrames to SQLite tables
coffee_df.to_sql('table1', conn, index=False, if_exists='replace')
meat_df.to_sql('table2', conn, index=False, if_exists='replace')
milk_df.to_sql('table3', conn, index=False, if_exists='replace')
df_co2.to_sql('table4', conn, index=False, if_exists='replace')

print(coffee_df.head(6))
print("Schema of table1:")
print(pd.read_sql_query("PRAGMA table_info(table1);", conn))
print("\nSchema of table2:")
print(pd.read_sql_query("PRAGMA table_info(table2);", conn))
print("\nSchema of table3:")
print(pd.read_sql_query("PRAGMA table_info(table3);", conn))
print("\nSchema of table4:")
print(pd.read_sql_query("PRAGMA table_info(table4);", conn))

query = "SELECT * FROM table1"
df = pd.read_sql_query(query, conn)
print(df)


      Country  Number of Bags  Bag Weight Harvest Year
0    Colombia               1        35.0  2021 / 2022
1      Taiwan               1        80.0  2021 / 2022
2        Laos              19        25.0  2021 / 2022
3  Costa Rica               1        22.0         2022
4    Colombia               2        24.0         2022
5   Guatemala               5        30.0         2022
Schema of table1:
   cid            name     type  notnull dflt_value  pk
0    0         Country     TEXT        0       None   0
1    1  Number of Bags  INTEGER        0       None   0
2    2      Bag Weight     REAL        0       None   0
3    3    Harvest Year     TEXT        0       None   0

Schema of table2:
   cid            name     type  notnull dflt_value  pk
0    0         Country     TEXT        0       None   0
1    1            Year  INTEGER        0       None   0
2    2  Production amt     REAL        0       None   0

Schema of table3:
   cid      name     type  notnull dflt_value  pk
0    

In [23]:

query = '''
SELECT table1.Country, table1."Bag Weight", table2."Production amt", table3."Milk amt", table4.co2_value
FROM table1
LEFT JOIN table2 ON table1.Country = table2.Country
LEFT JOIN table3 ON table1.Country = table3.Country
LEFT JOIN table4 ON table1.Country = table4.Country
'''

# Execute the query and load the result into a DataFrame
final_df = pd.read_sql_query(query, conn)
final_df.dropna(inplace=True)

# Close the connection
conn.close()

# Display the merged DataFrame
if final_df is not None:
    print(final_df)
    

           Country  Bag Weight  Production amt    Milk amt      co2_value
0         Colombia        35.0        442045.0   1762000.0   16409.825000
1         Colombia        35.0        442045.0   1762000.0   18217.656000
2         Colombia        35.0        442045.0   1762000.0   19442.434000
3         Colombia        35.0        442045.0   1762000.0   21257.599000
4         Colombia        35.0        442045.0   1762000.0   21708.640000
...            ...         ...             ...         ...            ...
28747462    Brazil        60.0      30397944.0  36667944.0  454230.010986
28747463    Brazil        60.0      30397944.0  36667944.0  456489.990234
28747464    Brazil        60.0      30397944.0  36667944.0  485339.996338
28747465    Brazil        60.0      30397944.0  36667944.0  486839.996338
28747466    Brazil        60.0      30397944.0  36667944.0  511619.995117

[28501440 rows x 5 columns]


In [24]:
#Perform Analysis
x= final_df[['Bag Weight', 'Production amt', 'Milk amt']]
y= final_df['co2_value']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(x_train, y_train)
print("Coefficient (Slope):", model.coef_)
print("Intercept:", model.intercept_)


Coefficient (Slope): [-2.51237496  0.00668882  0.00579052]
Intercept: 29994.873881135354
